# Trying to take apart the agents piece by piece

## Agent with Google Search

In [1]:
from langchain_community.llms.ollama import Ollama

from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.utilities.sql_database import SQLDatabase

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.chains.sql_database import query

from langchain.agents import Tool
from langchain import agents

from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 
from langchain.agents.agent_toolkits.sql import base

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

from langchain_community.retrievers.wikipedia import WikipediaRetriever

from langchain.globals import set_verbose
set_verbose(True)

In [2]:
res = WikipediaRetriever().invoke("Cars")
print(res[0].page_content)

c:\Projects\Python\text2sql\venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Projects\Python\text2sql\venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


A car, or an automobile, is a motor vehicle with wheels. Most definitions of cars state that they run primarily on roads, seat one to eight people, have four wheels, and mainly transport people, not cargo.French inventor Nicolas-Joseph Cugnot built the first steam-powered road vehicle in 1769, while French-born Swiss inventor François Isaac de Rivaz designed and constructed the first internal combustion-powered automobile in 1808. The modern car—a practical, marketable automobile for everyday use—was invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen. Commercial cars became widely available during the 20th century. One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced horse-drawn carriages. In Europe and other parts of the world, demand for automobiles did not increase until after World War II. The car is considered an essenti

In [3]:
template =\
"""Act as wikipedia advanced user. 

# Context
Use tools to answer: {tools}

# Tonality
Be positive, be funny, use emojies

# Format
Answer using following format:
Question: \"user question\"
Answer: \"your answer\"
Link: \"link to sources\"

Alwayes keep in mind what tool I should use: [{tool_names}]

Let\'s go:
Question: \"{input}\"
Throught: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

In [4]:
wiki_tool = Tool(
    name="wiki_tool",
    func=WikipediaRetriever().invoke,
    description="Input to this tool the theme of question. Output information from wikipedia page. Use this tool for search info."
)

In [5]:
agent = agents.create_react_agent(
    llm=Ollama(model="llama2:13b"),
    tools=[wiki_tool],
    prompt=prompt,
    output_parser=StrOutputParser()
)

In [6]:
res = agent.invoke({"input": "What is Cars? The movie or cartoon?", "intermediate_steps": ""})
print(res)

  Question: What is Cars? The movie or cartoon?

Answer: Oh, you want to know about Cars? Well, let me tell you - it's a classic! 😄 Both the movie and the cartoon are super popular, but if I had to choose, I'd say the movie is my favorite. 🎥 It was released in 2006, and it's an animated comedy-adventure film produced by Pixar Animation Studios and directed by John Lasseter. The movie follows the story of a hotshot race car named Lightning McQueen (voiced by Owen Wilson) who gets stranded in a small town on his way to a big race. 🏎️

But wait, there's more! The movie also features a cast of lovable characters, including Mater (voiced by Larry the Cable Guy), a rusty old tow truck who becomes Lightning's unlikely friend. And let's not forget about the beautiful scenery of Radiator Springs - it's like a real-life Disneyland for cars! 🌵

So, if you haven't seen Cars yet, you're missing out on some serious fun! 🎉 But don't worry, there are plenty of ways to catch up - you can stream the mov